In [1]:
import numpy as np
import ROOT

%jsroot on

In [2]:
%pwd

'/home/ihor/CLionProjects/DMolybTarget/analysis'

In [24]:
root_files_dir = "/home/ihor/CLionProjects/DMolybTarget/root_files/"
root_file_name = "MolybdenumProtonBombardmentAllEvents_v1.root"
tree_name = "neutrons"

In [25]:
root_file = ROOT.TFile.Open(root_files_dir + root_file_name)
if root_file.IsZombie():
    print("The input file does not filled with data")

In [26]:
root_file

In [27]:
dataframe = ROOT.RDataFrame(tree_name, root_files_dir + root_file_name)

In [28]:
dataframe

In [29]:
start, end = 0, 10
n_bins = 100
bin_width = (end - start) / n_bins
kinetic_energies = dataframe.Histo1D(("kinetic_energy", f"Born neutrons; E_kinetic [MeV]; Counts/{bin_width * 1e3} [keV]", n_bins, start, end) , "kinetic_energy")

In [30]:
c = ROOT.TCanvas()
kinetic_energies.Draw()
c.Draw()
# c.SaveAs("born_neutrons_ekin.svg")

In [44]:
type(kinetic_energies)

<class cppyy.gbl.ROOT.RDF.RResultPtr<TH1D> at 0x2ca0b970>

In [45]:
import numpy as np

In [67]:
bin_centers = []
bin_center_counts = []
bins = kinetic_energies.GetNbinsX()

for i in range(1, bins + 1):
    bin_center = kinetic_energies.GetBinCenter(i)
    # print(f"bin center: {bin_center}")
    bin_content = kinetic_energies.GetBinContent(i)
    # print(f"bin content: {bin_content}")
    bin_centers.append(bin_center)
    bin_center_counts.append(bin_content)

bin_centers = np.array(bin_centers)
bin_center_counts = np.array(bin_center_counts)

In [69]:
bin_centers

array([0.05, 0.15, 0.25, 0.35, 0.45, 0.55, 0.65, 0.75, 0.85, 0.95, 1.05,
       1.15, 1.25, 1.35, 1.45, 1.55, 1.65, 1.75, 1.85, 1.95, 2.05, 2.15,
       2.25, 2.35, 2.45, 2.55, 2.65, 2.75, 2.85, 2.95, 3.05, 3.15, 3.25,
       3.35, 3.45, 3.55, 3.65, 3.75, 3.85, 3.95, 4.05, 4.15, 4.25, 4.35,
       4.45, 4.55, 4.65, 4.75, 4.85, 4.95, 5.05, 5.15, 5.25, 5.35, 5.45,
       5.55, 5.65, 5.75, 5.85, 5.95, 6.05, 6.15, 6.25, 6.35, 6.45, 6.55,
       6.65, 6.75, 6.85, 6.95, 7.05, 7.15, 7.25, 7.35, 7.45, 7.55, 7.65,
       7.75, 7.85, 7.95, 8.05, 8.15, 8.25, 8.35, 8.45, 8.55, 8.65, 8.75,
       8.85, 8.95, 9.05, 9.15, 9.25, 9.35, 9.45, 9.55, 9.65, 9.75, 9.85,
       9.95])

In [70]:
bin_center_counts

array([1.3672e+04, 2.1087e+04, 2.3115e+04, 2.3273e+04, 2.2277e+04,
       2.0018e+04, 1.8148e+04, 1.6353e+04, 1.4527e+04, 1.3161e+04,
       1.1789e+04, 1.0975e+04, 9.8040e+03, 8.8910e+03, 8.3470e+03,
       7.4250e+03, 6.8330e+03, 6.1670e+03, 5.8360e+03, 5.1940e+03,
       4.6920e+03, 4.3370e+03, 3.9690e+03, 3.5070e+03, 3.2120e+03,
       2.8810e+03, 2.6750e+03, 2.5110e+03, 2.1550e+03, 1.9930e+03,
       1.7370e+03, 1.6510e+03, 1.4470e+03, 1.3160e+03, 1.1550e+03,
       1.1400e+03, 1.0260e+03, 9.3200e+02, 8.6000e+02, 7.6300e+02,
       6.8700e+02, 6.5400e+02, 5.8900e+02, 5.6900e+02, 4.8000e+02,
       4.4400e+02, 4.1700e+02, 3.8700e+02, 3.6400e+02, 3.5300e+02,
       3.1100e+02, 2.9900e+02, 2.7600e+02, 2.4300e+02, 2.2500e+02,
       2.0200e+02, 1.8700e+02, 2.0100e+02, 1.9200e+02, 1.8500e+02,
       1.6900e+02, 1.6400e+02, 1.6300e+02, 1.3400e+02, 1.2100e+02,
       1.1700e+02, 1.0500e+02, 1.0100e+02, 9.7000e+01, 9.3000e+01,
       8.9000e+01, 7.5000e+01, 6.2000e+01, 7.4000e+01, 5.7000e

In [101]:
import iminuit as im
from iminuit import Minuit
from iminuit.cost import LeastSquares

bin_center_counts_error = np.sqrt(bin_center_counts)
bin_center_counts_error[bin_center_counts_error == 0] = 1

def exponential(x, A, lambda_):
    return A * np.exp(-x / lambda_)

least_squares = LeastSquares(bin_centers, bin_center_counts, bin_center_counts_error, model=exponential)
m = Minuit(least_squares, A = 25.e3, lambda_=1.5)
# m.errordef = Minuit.LEAST_SQUARES

In [102]:
m.migrad()

┌─────────────────────────────────────────────────────────────────────────┐
│                                Migrad                                   │
├──────────────────────────────────┬──────────────────────────────────────┤
│ FCN = 1.485e+04 (χ²/ndof = 151.5)│              Nfcn = 46               │
│ EDM = 7.61e-06 (Goal: 0.0002)    │                                      │
├──────────────────────────────────┼──────────────────────────────────────┤
│          Valid Minimum           │   Below EDM threshold (goal x 10)    │
├──────────────────────────────────┼──────────────────────────────────────┤
│      No parameters at limit      │           Below call limit           │
├──────────────────────────────────┼──────────────────────────────────────┤
│             Hesse ok             │         Covariance accurate          │
└──────────────────────────────────┴──────────────────────────────────────┘
┌───┬─────────┬───────────┬───────────┬────────────┬────────────┬─────────┬─────────┬───────┐
│   │ Name    │   Value   │ Hesse Err │ Minos Err- │ Minos Err+ │ Limit-  │ Limit+  │ Fixed │
├───┼─────────┼───────────┼───────────┼────────────┼────────────┼─────────┼─────────┼───────┤
│ 0 │ A       │  25.85e3  │  0.06e3   │            │            │         │         │       │
│ 1 │ lambda_ │  1.1817   │  0.0018   │            │            │         │         │       │
└───┴─────────┴───────────┴───────────┴────────────┴────────────┴─────────┴─────────┴───────┘
┌─────────┬─────────────────────────┐
│         │           A     lambda_ │
├─────────┼─────────────────────────┤
│       A │    3.82e+03 -74.4686e-3 │
│ lambda_ │ -74.4686e-3    3.41e-06 │
└─────────┴─────────────────────────┘